# Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import pandas_datareader.data as web
import schwabdev
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta
%matplotlib inline

# Momentum Investing
- the strategy in investing in the stocks that increased in price the most

# Collecting S&P 500 constituents

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [4]:
sp500_constituents = sp500_constituents['Symbol'].to_list()

### changing format to match Schwab's

In [5]:
sp500_constituents = [x.replace('.', '/') for x in sp500_constituents]

# Setting up Schwab API

In [6]:
from key import api_key, api_secret

In [7]:
client = schwabdev.Client(app_key=api_key, app_secret=api_secret, capture_callback=False)

# Creating Datafram
- Basing this strategy in a 6 month change

In [8]:
my_columns = ['Ticker', 'Price (6 months)', 'Price (now)', 'Net Change', 'Number of Shares to Buy']

In [9]:
today = datetime.today()
six_months = today - relativedelta(months = 6)

In [10]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', period = 6, startDate=six_months, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_6_months = price_history['candles'][0]['close']
    net_change = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    entries.append(
        [
            symbol,
            price_6_months,
            price_now,
            net_change,
            'N/a'
        ]
    )

In [11]:
momentum_df = pd.DataFrame(entries, columns=my_columns)

### Sorting by biggest increases

In [12]:
momentum_df.sort_values(by='Net Change', ascending=False, inplace=True)

In [13]:
momentum_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
360,PLTR,44.86,93.40,108.20,N/a
308,MCK,507.41,700.70,38.09,N/a
171,EQT,37.48,51.61,37.70,N/a
468,VRSN,181.26,247.07,36.31,N/a
433,TTWO,161.79,214.44,32.54,N/a


# Creating Excel Output with all data

In [14]:
momentum_df_copy = momentum_df.copy()

In [15]:
momentum_df_copy = momentum_df_copy.drop('Number of Shares to Buy', axis=1)
momentum_df_copy.head()

,Ticker,Price (6 months),Price (now),Net Change
360,PLTR,44.86,93.40,108.20
308,MCK,507.41,700.70,38.09
171,EQT,37.48,51.61,37.70
468,VRSN,181.26,247.07,36.31
433,TTWO,161.79,214.44,32.54


In [40]:
writer = pd.ExcelWriter('total_data.xlsx', engine='xlsxwriter')

momentum_df_copy.to_excel(writer, sheet_name='Movers', index=False)

### Formatting

In [41]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [42]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [43]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying Formats

In [44]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format]
}

In [45]:
for column in column_formats.keys():
    writer.sheets['Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

In [46]:
writer.close()

# Keeping Top 50 Movers

In [23]:
final_df = momentum_df.copy()

In [24]:
final_df = final_df.head(50)
final_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 360 to 334
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   50 non-null     object 
 1   Price (6 months)         50 non-null     float64
 2   Price (now)              50 non-null     float64
 3   Net Change               50 non-null     float64
 4   Number of Shares to Buy  50 non-null     object 
dtypes: float64(3), object(2)
memory usage: 2.3+ KB


In [25]:
final_df.reset_index(inplace=True)
final_df.drop('index', axis = 1, inplace=True)
final_df

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,93.4000,108.20,N/a
1,MCK,507.41,700.7000,38.09,N/a
2,EQT,37.48,51.6100,37.70,N/a
3,VRSN,181.26,247.0700,36.31,N/a
4,TTWO,161.79,214.4400,32.54,N/a
5,PAYC,164.63,216.4200,31.46,N/a
6,NFLX,754.68,979.7050,29.82,N/a
7,TKO,115.25,149.2650,29.51,N/a
8,CNP,29.31,37.5550,28.13,N/a
9,DRI,159.92,202.7700,26.79,N/a


# Calculating the Number of Shares to Buy

In [26]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of the porfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('Invalid input! \n Please try again.')
        portfolio_size = input('Enter the size of the porfolio: ')


In [27]:
portfolio_input()

In [28]:
print(portfolio_size)

100000


In [29]:
position_size = float(portfolio_size) / len(final_df.index)
position_size

2000.0

In [30]:
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Price (now)'])

In [31]:
final_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,93.40,108.20,21
1,MCK,507.41,700.70,38.09,2
2,EQT,37.48,51.61,37.70,38
3,VRSN,181.26,247.07,36.31,8
4,TTWO,161.79,214.44,32.54,9


# Exporting to Excel

In [47]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_df.to_excel(writer, sheet_name='Recommended Trades', index=False)

In [48]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [49]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [50]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [51]:
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [52]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format],
    'E': ['Number of Shares to Buy', integer_format]
}

In [53]:
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [54]:
writer.close()